╔══<i><b>Alai-DeepLearning</b></i>════════════════════════════╗
###  &nbsp;&nbsp; **✎&nbsp;&nbsp;Week 13. RNN-Basis**
# Section 2. 1.Tensorflow을 활용한 RNN 구현하기(Option)


### _Objective_

1. 순환신경망의 기본적인 구조를 배우고 RNN을 작성합니다.

╚═════════════════════════════════════════╝

In [0]:
import tensorflow as tf
import numpy as np

"""
script 목적: 
    tensorflow 로 RNN 을 구현한다.
    hello 을 입력하면 ello<eos> 을 내보내는 모델을 생성한다.   
    이를 통해 시계열적인 문제를 풀어본다. 
    Many-to-Many 모델을 직접 생성해 본다.

data 처리: 문자열의 알파벳을 아래와 같이 표현한다.  
    h = [1, 0, 0, 0, 0]
    e = [0, 1, 0, 0, 0]
    l = [0, 0, 1, 0, 0]
    o = [0, 0, 0, 1, 0]
    <eos> = [0, 0, 0, 0, 1]
"""

# data 
h = [1, 0, 0, 0, 0]
e = [0, 1, 0, 0, 0]
l = [0, 0, 1, 0, 0]
o = [0, 0, 0, 1, 0]
eos = [0, 0, 0, 0, 1]
train_xs = np.asarray([h, e, l, l, o, eos])
train_xs = np.expand_dims(train_xs, axis=0)
n_classes = 5

# xs
time_step = 6

tf.reset_default_graph()
xs = tf.placeholder(dtype=tf.float32, shape=[None, time_step, n_classes])
trans_xs = tf.transpose(xs, perm=[1, 0, 2])
unstacked_xs = tf.unstack(trans_xs)

# units
n_units = 10

# Generate Weights and Bias
tf.random.set_random_seed(1)
w_hy = tf.Variable(tf.random.normal([n_units, n_classes], 0.1), name='output_w')
w_hs = tf.Variable(tf.random.normal([n_units, n_units], 0.1), name='state_w')
w_hx = tf.Variable(tf.random.normal([n_classes, n_units], 0.1), name='input_w')

b_hx = tf.Variable(tf.zeros([n_units]), name='input_b')
b_hy = tf.Variable(tf.zeros([n_classes]), name='output_b')

# Build RNN Model
state_layer = tf.constant(0.)
for ind, x in enumerate(unstacked_xs[:-1]):
    with tf.name_scope('input'):
        # input layer
        input_layer = tf.matmul(x, w_hx) + b_hx
        input_layer = input_layer + state_layer
        input_layer = tf.nn.relu(input_layer)
        input_layer = tf.identity(input_layer, 'input_{}'.format(ind + 1))

    with tf.name_scope('output'):
        # output layer
        output_layer = tf.matmul(input_layer, w_hy) + b_hy
        output_layer = tf.nn.relu(output_layer)
        output_layer = tf.identity(output_layer, 'output_{}'.format(ind + 1))
        tf.add_to_collection('outputs', output_layer)

    with tf.name_scope('state'):
        # state layer
        state_layer = tf.matmul(input_layer, w_hs)
        state_layer = tf.identity(state_layer, 'state_{}'.format(ind + 1))

    with tf.name_scope('loss'):
        y = unstacked_xs[ind+1: ind+2]
        loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=output_layer)
        tf.add_to_collection('loss', loss)

# Loss
with tf.name_scope('mean_loss'):
    loss = tf.reduce_mean(tf.get_collection('loss'), name='mean_loss')

# train Optimizer
train_op = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(loss)

# Session open
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# training
max_step = 5000
for i in range(max_step):
    loss_, _ = sess.run([loss, train_op], {xs: train_xs})
    print(loss_)




In [0]:
predict = sess.run(tf.get_collection('outputs'), {xs: train_xs})
predict=np.asarray(predict)
print(predict)

In [0]:
char2num = {0:'h', 1:'e', 2:'l', 3:'o', 4:'<eos>'}    

predict = sess.run(tf.get_collection('outputs'), {xs: train_xs})
predict = np.asarray(predict)
predict = np.squeeze(predict)
# onehot2classes
classes = np.argmax(predict, axis=1)

# char2string
str_ = ''
str_ = ''.join([char2num[c] for c in classes.tolist()])
print(str_)

In [0]:
from IPython.display import clear_output, Image, display, HTML
import numpy as np    

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

# Usage:
# show_graph(tf.get_default_graph())

In [0]:
show_graph(tf.get_default_graph())